In [2]:
import pandas as pd
import numpy as np
from fragmentembedding import FragmentEmbedder
from rdkit import Chem

/var/folders/y_/2d6q3v617qj78nxjx8b1_2wc0000gn/T/ipykernel_8840/3751962780.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
def is_valid_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return mol is not None

In [3]:
with open("../DUD-E/cah2/actives_final.ism", "r") as file:
    txt = file.read()
    lines = txt.split("\n")
    active_smiles = [line.split(" ")[0] for line in lines]

In [4]:
with open("../DUD-E/cah2/decoys_final.ism", "r") as file:
    txt = file.read()
    lines = txt.split("\n")
    decoy_smiles = [line.split(" ")[0] for line in lines]

In [5]:
total_smiles = active_smiles+decoy_smiles

In [6]:
len(total_smiles)

31663

In [11]:
smiles_list = [
   "c1ccccc1S=N",               # Phenyl sulfurimidous acid
    "c1ccccc1NS(=O)(=O)O",       # N-Phenyl sulfamic acid
    "c1ccccc1S(=O)(=O)N",        # Phenyl sulfonamide
    "c1ccccc1NS(=O)(=O)N",       # N-Phenyl sulfamide
    "c1ccccc1NC=O",              # N-Phenylformamide
    "c1ccccc1NC(=O)N",           # N-Phenylurea
    "c1ccccc1S(=O)(=O)c2ccccc2", # Diphenyl sulfone
    "c1ccccc1OS(=O)(=O)Oc2ccccc2", # Diphenyl sulfate
    "c1ccccc1NS"
]

In [3]:
smiles_list = [
    'Cc2ccc(C(=O)CNc1ccc(S(=O)O)cc1)cc2',          # sulfurimidous acid (approximate)
    'Cc2ccc(C(=O)CNc1ccc(S(=O)(=O)O)cc1)cc2',   # sulfamic acid
    'Cc2ccc(C(=O)CNc1ccc(S(N)(=O)=O)cc1)cc2',   # sulfonamide
    'Cc2ccc(C(=O)CNc1ccc(NS(N)(=O)=O)cc1)cc2',   # sulfamide
    'Cc2ccc(C(=O)CNc1ccc(C(N)=O)cc1)cc2',        # formamide
    'Cc2ccc(C(=O)CNc1ccc(NC(N)=O)cc1)cc2',     # urea
    'Cc2ccc(C(=O)CNc1ccc(S(C)(=O)=O)cc1)cc2',   # dimethyl sulfone
    'COS(=O)(=O)Oc2ccc(NCC(=O)c1ccc(C)cc1)cc2', # dimethyl sulfate
    'Cc2ccc(C(=O)CNc1ccc(SN)cc1)cc2'             # thiohydroxylamine (approximate)
]

In [4]:
smiles_list = [
    'Cc2ccc(C(=O)CNc1ccc(S(N)(=O)=O)cc1)cc2',   # sulfonamide
    "Cc2ccc(C(=O)CNc1ccc(C(=O)N)cc1)cc2",             # Formamide
    "Cc2ccc(C(=O)CNc1ccc(S(=O)(=O)[O-])cc1)cc2",       # Sulfonate
    "Cc2ccc(C(=O)CNc1ccc(S(=O)[O-])cc1)cc2",           # Sulfinate
    "Cc2ccc(C(=O)CNc1ccc(S[O-])cc1)cc2",               # Sulfenate
    "Cc2ccc(C(=O)CNc1ccc(S(=O)N)cc1)cc2",              # Sulfinamide
    "Cc2ccc(C(=O)CNc1ccc(SN)cc1)cc2",                  # Thiohydroxylamine
    "Cc2ccc(C(=O)CNc1ccc(S(=O)(=O)C)cc1)cc2",          # Methyl sulfone
    "Cc2ccc(C(=O)CNc1ccc(S(=O)C)cc1)cc2",              # Methyl sulfoxide
    "Cc2ccc(C(=O)CNc1ccc(S)cc1)cc2"                    # Thiol
]

In [3]:
depmap = pd.read_excel("../DepMap/filtered_depmap.xlsx")

In [5]:
smiles_list = depmap["SMILES"]

In [5]:
total_smiles=smiles_list

In [6]:
invalid = []
for i, val in enumerate(total_smiles):
    if not is_valid_smiles(val):
        invalid.append(i)
invalid

[]

In [7]:
block = 1000
invalid_blocks = set()
for inv in invalid:
    invalid_blocks.add(inv//block)
invalid_blocks

set()

In [8]:
def iterate_block(fe, index):
    X = []
    for i in range(index*block, min((index+1)*block, len(total_smiles))):
        if i in invalid:
            continue
        try:
            Y = fe.transform([total_smiles[i]])
            X.append(Y)
        except ValueError:
            invalid.append(i)
    return np.concatenate(X)

In [9]:
fe = FragmentEmbedder()
X = []
for i in range(0, (len(total_smiles)//block)+1):
    print(block*i)
    if i in invalid_blocks:
        X.append(iterate_block(fe, i))
    else:
        try:
            X.append(fe.transform(total_smiles[block*i:block*(i+1)]))
        except ValueError:
            X.append(iterate_block(fe, i))
final = np.concatenate(X)
final.shape

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator VarianceThreshold from version 0.23.2 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


0


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator VarianceThreshold from version 1.2.2 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator KBinsDiscretizer from version 1.2.2 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


(10, 512)

In [11]:
invalid

[]

In [15]:
fe = FragmentEmbedder()
X = fe.transform(filtered_dep_smiles)
X.shape

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator VarianceThreshold from version 0.23.2 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator VarianceThreshold from version 1.2.2 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning

ValueError: Input X contains infinity or a value too large for dtype('float32').

In [13]:
np.save("full_depmap_embed.npy", final)

In [12]:
np.save("new_small_embeddings.npy", final)